In [1]:
!wget https://raw.githubusercontent.com/cblancac/SentimentAnalysisBert/main/data/train_150k.txt
!wget https://raw.githubusercontent.com/cblancac/SentimentAnalysisBert/main/data/test_62k.txt

--2024-02-03 19:39:08--  https://raw.githubusercontent.com/cblancac/SentimentAnalysisBert/main/data/train_150k.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11697788 (11M) [text/plain]
Saving to: ‘train_150k.txt.1’

train_150k.txt.1     17%[==>                 ]   1.95M   309KB/s    eta 24s    ^C
--2024-02-03 19:39:14--  https://raw.githubusercontent.com/cblancac/SentimentAnalysisBert/main/data/test_62k.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4839762 (4.6M) [text/plain]
Saving to: ‘test_62k.txt.1’

test_62k.txt.1

In [1]:
import pandas as pd
from datasets import Dataset
data = pd.read_csv('train_150k.txt', sep='\t', header=None, names=['label', 'text'])
data = data[['text', 'label']].to_dict('list')
train_dataset = Dataset.from_dict(data)

data = pd.read_csv('test_62k.txt', sep='\t', header=None, names=['label', 'text'])
data = data[['text', 'label']].to_dict('list')
eval_dataset = Dataset.from_dict(data)

/home/chirag/sample/sample/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained BERT model and tokenizer
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = train_dataset.map(lambda examples: bert_tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True)
eval_dataset = eval_dataset.map(lambda examples: bert_tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 61998/61998 [00:15<00:00, 3902.33 examples/s]


In [3]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

# Initialize the trainer
trainer = Trainer(
    model=bert_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                       # training arguments, defined above
    train_dataset=train_dataset,                    # training dataset
    )

trainer.train()

Step,Training Loss
500,0.477700
1000,0.389700
1500,0.372400
2000,0.362100
2500,0.325100
3000,0.267700
3500,0.265600
4000,0.263500
4500,0.261700
5000,0.188400


TrainOutput(global_step=7032, training_loss=0.2676658528317635, metrics={'train_runtime': 11166.5428, 'train_samples_per_second': 40.295, 'train_steps_per_second': 0.63, 'total_flos': 1.183881349145088e+17, 'train_loss': 0.2676658528317635, 'epoch': 3.0})

In [6]:
trainer.save_model('fineBERT')

In [45]:
data = pd.read_csv('test_62k.txt', sep='\t', header=None, names=['label', 'text'])
data = data.iloc[41600:42700]
data = data[['text', 'label']].to_dict('list')
ev_dataset = Dataset.from_dict(data)
ev_dataset = ev_dataset.map(lambda examples: bert_tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True)
ev_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
# trainer1 = Trainer(model = BertForSequenceClassification.from_pretrained('fineBERT'), )
# test_metrics = trainer.evaluate(ev_dataset)
trainer1 = Trainer(
    model=BertForSequenceClassification.from_pretrained('fineBERT'),
    args=training_args,
    train_dataset=train_dataset,  # replace with your actual training dataset
    eval_dataset=ev_dataset,
)
results = trainer1.evaluate()
results

Map: 100%|██████████| 1100/1100 [00:00<00:00, 4152.90 examples/s]


{'eval_loss': 0.45738670229911804,
 'eval_runtime': 8.2189,
 'eval_samples_per_second': 133.838,
 'eval_steps_per_second': 2.19}

In [46]:
from torch.utils.data import DataLoader

In [47]:
dataloader = DataLoader(ev_dataset, batch_size=5)
fine_tuned_model = BertForSequenceClassification.from_pretrained('fineBERT')
fine_tuned_model.eval()
# Perform inference and calculate accuracy
correct_predictions = 0
total_samples = 0

for batch in dataloader:
    with torch.no_grad():
        input_ids = [ids.to(device) for ids in batch['input_ids']]
        attention_mask = [mask.to(device) for mask in batch['attention_mask']]
        labels = batch['label'].to(device)

        # Ensure input_ids and attention_mask are not lists
        input_ids = torch.stack(input_ids)
        attention_mask = torch.stack(attention_mask)

        # Make sure the model is on the same device
        fine_tuned_model = fine_tuned_model.to(device)

        outputs = fine_tuned_model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        correct_predictions += (predictions == labels).sum().item()
        total_samples += len(labels)

test_accuracy = correct_predictions / total_samples
print(f'Test Accuracy on the New Dataset: {test_accuracy}')

Test Accuracy on the New Dataset: 0.8536363636363636
